<a href="https://colab.research.google.com/github/vrushabhmudda/Deep-Learning-project/blob/main/Inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from io import BytesIO
from PIL import Image
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

# =====================================
# MOUNT GOOGLE DRIVE
# =====================================
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

# =====================================
# DEFINE PATHS
# =====================================
DATA_PATH = '/content/drive/MyDrive/ISIC_2024_Project/data/raw/'
ZIP_PATH = '/content/drive/MyDrive/isic-2024-challenge.zip'
METADATA_FILE = DATA_PATH + 'train-metadata.csv'
HDF5_PATH = DATA_PATH + 'train-image.hdf5'
MODEL_SAVE_PATH = '/content/drive/MyDrive/ISIC_2024_Project/models/'
RESULTS_PATH = '/content/drive/MyDrive/ISIC_2024_Project/results/'

# =====================================
# CHECK AND EXTRACT DATA
# =====================================
# Create directories if they don't exist
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Check if metadata exists, if not extract from zip
if not os.path.exists(METADATA_FILE):
    print(f"\n⚠️ Metadata file not found. Extracting from {ZIP_PATH}...")
    if os.path.exists(ZIP_PATH):
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            extract_path = '/content/drive/MyDrive/ISIC_2024_Project/data/'
            zip_ref.extractall(extract_path)
            print(f"   ✅ Extraction complete to {extract_path}")
    else:
        raise FileNotFoundError(f"❌ Zip file not found at {ZIP_PATH}")
else:
    print(f"✅ Data files found")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted
✅ Data files found


In [ ]:
# =====================================
# LOAD METADATA
# =====================================
print("\n📊 Loading metadata...")
train_metadata = pd.read_csv(METADATA_FILE)
print(f"   Metadata shape: {train_metadata.shape}")
print(f"   Columns: {list(train_metadata.columns)}")

# Check target distribution
if 'target' in train_metadata.columns:
    target_counts = train_metadata['target'].value_counts()
    print(f"\n   Target distribution:")
    print(f"   - Benign (0): {target_counts.get(0, 0):,}")
    print(f"   - Malignant (1): {target_counts.get(1, 0):,}")
    if target_counts.get(1, 1) > 0:
        print(f"   - Imbalance ratio: 1:{target_counts.get(0, 0) // target_counts.get(1, 1)}")

# =====================================
# CUSTOM HDF5 DATA GENERATOR
# =====================================
class HDF5ImageGenerator(keras.utils.Sequence):
    """Memory-efficient generator that loads images from HDF5 in batches"""

    def __init__(self, h5_path, image_ids, labels, batch_size=32,
                 target_size=(299, 299), augment=False, shuffle=True):
        self.h5_path = h5_path
        self.image_ids = image_ids
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.augment = augment
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_ids))

        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = min((index + 1) * self.batch_size, len(self.image_ids))
        batch_indexes = self.indexes[start_idx:end_idx]

        X, y = self.__data_generation(batch_indexes)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_indexes):
        X = np.empty((len(batch_indexes), *self.target_size, 3), dtype=np.float32)
        y = np.empty((len(batch_indexes)), dtype=np.float32)

        with h5py.File(self.h5_path, 'r') as h5_file:
            for i, idx in enumerate(batch_indexes):
                img_id = self.image_ids[idx]

                jpeg_bytes = h5_file[img_id][()]
                img = Image.open(BytesIO(jpeg_bytes))
                img = img.resize(self.target_size, Image.BILINEAR)
                img_array = np.array(img, dtype=np.float32)

                if len(img_array.shape) == 2:
                    img_array = np.stack([img_array] * 3, axis=-1)

                img_array = (img_array / 127.5) - 1.0

                X[i,] = img_array
                y[i] = self.labels[idx]

        if self.augment:
            X = self.__augment_batch(X)

        return X, y

    def __augment_batch(self, images):
        augmented = []
        for img in images:
            if np.random.rand() > 0.5:
                img = np.fliplr(img)
            if np.random.rand() > 0.5:
                img = np.flipud(img)
            augmented.append(img)
        return np.array(augmented)




📊 Loading metadata...


/tmp/ipython-input-79945172.py:5: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(METADATA_FILE)


   Metadata shape: (401059, 55)
   Columns: ['isic_id', 'target', 'patient_id', 'age_approx', 'sex', 'anatom_site_general', 'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm', 'tbp_lv_dnn_lesion_confidence']

   Target di

In [ ]:
# =====================================
# SETUP DATA GENERATORS
# =====================================
print("\n🔧 Setting up data generators...")

train_ids = train_metadata['isic_id'].values
train_labels = train_metadata['target'].values

train_ids_split, val_ids_split, train_labels_split, val_labels_split = train_test_split(
    train_ids, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

print(f"   Training samples: {len(train_ids_split):,}")
print(f"   Validation samples: {len(val_ids_split):,}")
print(f"   Malignant in train: {np.sum(train_labels_split):,}")
print(f"   Malignant in val: {np.sum(val_labels_split):,}")

BATCH_SIZE = 32
IMG_SIZE = (299, 299)

train_generator = HDF5ImageGenerator(
    h5_path=HDF5_PATH,
    image_ids=train_ids_split,
    labels=train_labels_split,
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    augment=True,
    shuffle=True
)

val_generator = HDF5ImageGenerator(
    h5_path=HDF5_PATH,
    image_ids=val_ids_split,
    labels=val_labels_split,
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    augment=False,
    shuffle=False
)

print("✅ Generators ready!")

# =====================================
# BUILD INCEPTIONV3 MODEL
# =====================================
print("\n🏗️ Building InceptionV3 model...")

base_model = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dense(512, activation='relu', name='fc1')(x)
x = Dropout(0.5, name='dropout')(x)
predictions = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')  # This is Sensitivity
    ]
)

print(f"   Total params: {model.count_params():,}")
print("✅ Model built and compiled!")


🔧 Setting up data generators...
   Training samples: 320,847
   Validation samples: 80,212
   Malignant in train: 314
   Malignant in val: 79
✅ Generators ready!

🏗️ Building InceptionV3 model...
   Total params: 22,852,385
✅ Model built and compiled!


In [ ]:
# =====================================
# CALCULATE CLASS WEIGHTS
# =====================================
print("\n⚖️ Calculating class weights...")

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels_split),
    y=train_labels_split
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"   Class 0 (Benign) weight: {class_weight_dict[0]:.4f}")
print(f"   Class 1 (Malignant) weight: {class_weight_dict[1]:.4f}")

# =====================================
# SETUP CALLBACKS
# =====================================
print("\n📋 Setting up callbacks...")

callbacks = [
    keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH + 'inceptionv3_best.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=5,
        verbose=1,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=1e-7
    ),
    keras.callbacks.CSVLogger(
        RESULTS_PATH + 'training_log.csv',
        append=False
    )
]

print("✅ Callbacks configured!")





⚖️ Calculating class weights...
   Class 0 (Benign) weight: 0.5005
   Class 1 (Malignant) weight: 510.9029

📋 Setting up callbacks...
✅ Callbacks configured!


In [ ]:
# =====================================
# TRAIN MODEL
# =====================================
print("\n🚀 Starting training...\n")

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

# =====================================
# SAVE FINAL MODEL TO GOOGLE DRIVE
# =====================================
print("\n💾 Saving model to Google Drive...")
model.save(MODEL_SAVE_PATH + 'inceptionv3_final.h5')
print(f"   ✅ Model saved to {MODEL_SAVE_PATH}inceptionv3_final.h5")


🚀 Starting training...



/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.7321 - auc: 0.5372 - loss: 2.7511 - precision: 0.0013 - recall: 0.3397
Epoch 1: val_auc improved from -inf to 0.77520, saving model to /content/drive/MyDrive/ISIC_2024_Project/models/inceptionv3_best.h5


10027/10027 ━━━━━━━━━━━━━━━━━━━━ 1863s 184ms/step - accuracy: 0.7321 - auc: 0.5372 - loss: 2.7510 - precision: 0.0013 - recall: 0.3397 - val_accuracy: 0.0234 - val_auc: 0.7752 - val_loss: 1.4627 - val_precision: 0.0010 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/15
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.4940 - auc: 0.6460 - loss: 0.9011 - precision: 0.0014 - recall: 0.6748
Epoch 2: val_auc did not improve from 0.77520
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 1826s 182ms/step - accuracy: 0.4940 - auc: 0.6460 - loss: 0.9012 - precision: 0.0014 - recall: 0.6748 - val_accuracy: 0.6631 - val_auc: 0.7717 - val_loss: 0.8973 - val_precision: 0.0023 - val_recall: 0.7848 - learning_rate: 0.0010
Epoch 3/15
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.5823 - auc: 0.6842 - loss: 1.0577 - precision: 0.0017 - recall: 0.6783
Epoch 3: val_auc did not improve from 0.77520
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 1802s 180ms/step - accuracy: 0.5823 - auc: 0.6842 - loss: 1.0


Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 1815s 181ms/step - accuracy: 0.6512 - auc: 0.7516 - loss: 0.7189 - precision: 0.0022 - recall: 0.7104 - val_accuracy: 0.8436 - val_auc: 0.7995 - val_loss: 0.3705 - val_precision: 0.0037 - val_recall: 0.5949 - learning_rate: 0.0010
Epoch 7/15
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.7247 - auc: 0.7630 - loss: 0.6388 - precision: 0.0022 - recall: 0.6573
Epoch 7: val_auc did not improve from 0.79946
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 1832s 183ms/step - accuracy: 0.7247 - auc: 0.7630 - loss: 0.6389 - precision: 0.0022 - recall: 0.6573 - val_accuracy: 0.8828 - val_auc: 0.7875 - val_loss: 0.3655 - val_precision: 0.0049 - val_recall: 0.5823 - learning_rate: 5.0000e-04
Epoch 8/15
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.6338 - auc: 0.7445 - loss: 0.6750 - precision: 0.0022 - recall: 0.7579
Epoch 8: val_auc did not improve from 0.79946

Epoch 8: Red


✅ Training complete!

💾 Saving model to Google Drive...
   ✅ Model saved to /content/drive/MyDrive/ISIC_2024_Project/models/inceptionv3_final.h5


In [ ]:
# =====================================
# EVALUATION ON VALIDATION SET
# =====================================
print("\n📊 Evaluating model on validation set...")

# Get predictions
y_pred_probs = model.predict(val_generator, verbose=1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
y_true = val_labels_split

# Calculate metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)  # This is SENSITIVITY
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_probs)

# Calculate specificity
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn + fp)

print("\n📈 EVALUATION METRICS:")
print(f"   Accuracy: {accuracy:.4f}")
print(f"   Sensitivity (Recall): {recall:.4f}")
print(f"   Specificity: {specificity:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   F1 Score: {f1:.4f}")
print(f"   AUC-ROC: {roc_auc:.4f}")

# Save metrics to CSV
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Sensitivity (Recall)', 'Specificity', 'Precision', 'F1 Score', 'AUC-ROC'],
    'Value': [accuracy, recall, specificity, precision, f1, roc_auc]
})
metrics_df.to_csv(RESULTS_PATH + 'evaluation_metrics.csv', index=False)
print(f"\n✅ Metrics saved to {RESULTS_PATH}evaluation_metrics.csv")

# =====================================
# PLOT AND SAVE CONFUSION MATRIX
# =====================================
print("\n📊 Generating confusion matrix...")

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Benign', 'Malignant'],
            yticklabels=['Benign', 'Malignant'])
plt.title('Confusion Matrix - InceptionV3', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(RESULTS_PATH + 'confusion_matrix.png', dpi=300, bbox_inches='tight')
print(f"   ✅ Confusion matrix saved to {RESULTS_PATH}confusion_matrix.png")
plt.close()


📊 Evaluating model on validation set...
2507/2507 ━━━━━━━━━━━━━━━━━━━━ 331s 129ms/step

📈 EVALUATION METRICS:
   Accuracy: 0.8436
   Sensitivity (Recall): 0.5949
   Specificity: 0.8438
   Precision: 0.0037
   F1 Score: 0.0074
   AUC-ROC: 0.7996

✅ Metrics saved to /content/drive/MyDrive/ISIC_2024_Project/results/evaluation_metrics.csv

📊 Generating confusion matrix...
   ✅ Confusion matrix saved to /content/drive/MyDrive/ISIC_2024_Project/results/confusion_matrix.png


In [ ]:
# =====================================
# PLOT AND SAVE ROC CURVE
# =====================================
print("\n📈 Generating ROC curve...")

fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
roc_auc_value = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_value:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12)
plt.title('ROC Curve - InceptionV3', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_PATH + 'roc_curve.png', dpi=300, bbox_inches='tight')
print(f"   ✅ ROC curve saved to {RESULTS_PATH}roc_curve.png")
plt.close()

# =====================================
# PLOT AND SAVE TRAINING HISTORY
# =====================================
print("\n📉 Generating training history plots...")

# Plot 1: Loss curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Loss', fontsize=11)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Accuracy', fontsize=11)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(alpha=0.3)

# AUC
axes[1, 0].plot(history.history['auc'], label='Training AUC', linewidth=2)
axes[1, 0].plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
axes[1, 0].set_title('Model AUC', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('AUC', fontsize=11)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(alpha=0.3)

# Recall (Sensitivity)
axes[1, 1].plot(history.history['recall'], label='Training Recall', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Validation Recall', linewidth=2)
axes[1, 1].set_title('Model Recall (Sensitivity)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch', fontsize=11)
axes[1, 1].set_ylabel('Recall', fontsize=11)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_PATH + 'training_history.png', dpi=300, bbox_inches='tight')
print(f"   ✅ Training history saved to {RESULTS_PATH}training_history.png")
plt.close()



📈 Generating ROC curve...
   ✅ ROC curve saved to /content/drive/MyDrive/ISIC_2024_Project/results/roc_curve.png

📉 Generating training history plots...
   ✅ Training history saved to /content/drive/MyDrive/ISIC_2024_Project/results/training_history.png


In [ ]:
# =====================================
# SAVE DETAILED CLASSIFICATION REPORT
# =====================================
print("\n📄 Generating classification report...")

report = classification_report(y_true, y_pred,
                               target_names=['Benign', 'Malignant'],
                               output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(RESULTS_PATH + 'classification_report.csv')
print(f"   ✅ Classification report saved to {RESULTS_PATH}classification_report.csv")

# =====================================
# FINAL SUMMARY
# =====================================
print("\n" + "="*60)
print("📊 FINAL SUMMARY")
print("="*60)
print(f"\n✅ Model saved to: {MODEL_SAVE_PATH}")
print(f"✅ All evaluation metrics and plots saved to: {RESULTS_PATH}")
print("\nFiles saved:")
print(f"   1. inceptionv3_best.h5 (best model during training)")
print(f"   2. inceptionv3_final.h5 (final model)")
print(f"   3. evaluation_metrics.csv")
print(f"   4. confusion_matrix.png")
print(f"   5. roc_curve.png")
print(f"   6. training_history.png")
print(f"   7. training_log.csv")
print(f"   8. classification_report.csv")
print("\n" + "="*60)
print("\n🎉 All done! Ready for presentation.")
print("="*60)


📄 Generating classification report...
   ✅ Classification report saved to /content/drive/MyDrive/ISIC_2024_Project/results/classification_report.csv

📊 FINAL SUMMARY

✅ Model saved to: /content/drive/MyDrive/ISIC_2024_Project/models/
✅ All evaluation metrics and plots saved to: /content/drive/MyDrive/ISIC_2024_Project/results/

Files saved:
   1. inceptionv3_best.h5 (best model during training)
   2. inceptionv3_final.h5 (final model)
   3. evaluation_metrics.csv
   4. confusion_matrix.png
   5. roc_curve.png
   6. training_history.png
   7. training_log.csv
   8. classification_report.csv


🎉 All done! Ready for presentation.
